In [ ]:
from torch import device, cuda
import os
import torch
from tqdm import tqdm
import pandas as pd
import logging

from data_parser import adj_matrix, nodes
from models import CombinedModel, RETINA_MODEL
from image_parser import test_loader
from model_config_manager import ModelConfigManager

import seaborn as sns
import matplotlib.pyplot as plt

dev = device("cuda" if cuda.is_available() else "cpu")

logging.basicConfig(
    filename="test_log.log", 
    level=logging.INFO, 
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("test_log")

In [ ]:
# Create the ModelConfigManager and load configurations from YAML files
config_manager = ModelConfigManager()

# Get a specific configuration by model name
config_manager.set_model_config(RETINA_MODEL)

combined_model = CombinedModel(adj_matrix, neurons=nodes, model_config=config_manager.model_config)
combined_model.load_state_dict(torch.load(os.path.join("models", "model_vgg16_more_images_10_epochs.pth")), strict=False)
combined_model = combined_model.to(dev)
_ = combined_model.eval()

In [ ]:
# Test de model
correct = 0
total = 0
test_results_df = pd.DataFrame(columns=["Image", "Real Label", "Predicted Label", "Correct Prediction"])

j = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(dev), labels.to(dev)
        outputs = combined_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        # Convert the tensor values to CPU and numpy
        labels_cpu = labels.cpu().numpy()
        predicted_cpu = predicted.cpu().numpy()
        
        # Check if the prediction is correct
        correct_predictions = (predicted == labels)
        correct_cpu = correct_predictions.cpu().numpy()
        
        image_names = [a[0] for a in test_loader.dataset.dataset.samples[j * test_loader.batch_size: (j + 1) * test_loader.batch_size]]
        j += 1
        
        batch_df = pd.DataFrame({
            "Image": image_names,
            "Real Label": labels_cpu,
            "Predicted Label": predicted_cpu,
            "Correct Prediction": correct_cpu.astype(int)
        })
        
        # Append the batch DataFrame to the list
        test_results_df = pd.concat([test_results_df, batch_df], ignore_index=True)
        
logger.info(f"Accuracy on the {total} test images: {100 * correct / total}%")

In [ ]:
# Calculate the percentage of correct answers for each Weber ratio
test_results_df['yellow'] = test_results_df['Image'].apply(lambda x: x.split('_')[1])
test_results_df['blue'] = test_results_df['Image'].apply(lambda x: x.split('_')[2])
test_results_df['weber_ratio'] = test_results_df.apply(lambda row: max(int(row['yellow']), int(row['blue'])) / min(int(row['yellow']), int(row['blue'])), axis=1)
correct_percentage = test_results_df.groupby('weber_ratio')['Correct Prediction'].mean() * 100

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=correct_percentage.index, y=correct_percentage.values)
plt.xlabel('Weber Ratio')
plt.ylabel('Percentage of Correct Answers')
plt.title('Percentage of Correct Answers for Each Weber Ratio')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()